In [1]:
import os

In [2]:
%pwd

'c:\\Users\\anujs\\Documents\\Chicken-Disease-Classification-Project\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'c:\\Users\\anujs\\Documents\\Chicken-Disease-Classification-Project'

In [5]:
import tensorflow as tf

c:\Users\anujs\Documents\Chicken-Disease-Classification-Project\classy\Lib\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


In [6]:
model = tf.keras.models.load_model('artifacts/training/model.h5')

In [ ]:
from dataclasses import dataclass
import pathlib as Path
@dataclass(frozen = True)
class EvaluationConfig:
    path_of_model: Path
    training_data :Path
    all_params :dict
    params_image_size:list
    params_batch_size:int

In [ ]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories , save_json

In [10]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_validation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model = "artifacts/training/model.h5",
            training_data = "artifacts/data_ingestion/chicken-fecal-images",
            all_params = self.params,
            params_image_size = self.params.IMAGE_SIZE,
            params_batch_size = self.params.BATCH_SIZE
        )
        return eval_config
    

In [11]:
from urllib.parse import urlparse

In [ ]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config
    
    def _valid_generator(self):
        
        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split = 0.20
        )

        dataflow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size,
            interpolation = "bilinear"
        )
        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self._valid_generator = valid_datagenerator.flow_from_directory(
            directory = self.config.training_data,
            subset = "validation",
            shuffle = False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path:Path):
        model = tf.keras.models.load_model(path)
        return model
    
    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self._valid_generator)

    def save_score(self):
        scores = {"loss":self.score[0],"accuracy":self.score[1]}
        save_json(path = Path("score.json"),data = scores)

In [16]:
try:
    config = ConfigurationManager()
    val_config = config.get_validation_config()
    evaluat = Evaluation(val_config)
    evaluat.evaluation()
    evaluat.save_score()

except Exception as e:
    raise e

Found 116 images belonging to 2 classes.
8/8 ━━━━━━━━━━━━━━━━━━━━ 35s 4s/step - accuracy: 0.9483 - loss: 0.1589
